# Implementation of Ant Colony Optimization (ACO)
## Implemented by: Nishchal Gaba(nishgaba9@gmail.com), Rohit Tyagi(rohittyagi295@gmail.com)
## GitHub: https://github.com/nishgaba-ai/Python-self 
### REFERENCES:
#### 1> Wang, J., Cao, J., Li, B., Lee, S., & Sherratt, R. S. (2015). Bio-inspired ant colony optimization based clustering algorithm with mobile sinks for applications in consumer home automation networks. IEEE Transactions on Consumer Electronics, 61(4), 438-444.
#### 2> Aco-pants package (https://pypi.python.org/pypi/ACO-Pants)

### 1. Import Files

In [115]:
# Import files
import os
import sys
import numpy as np
import matplotlib as plt
import tensorflow as tf
import time
import random
import math
import pandas as pd
import sklearn
from scipy import misc
import glob
import pickle
import pants
from sklearn.cluster import KMeans
import collections
%matplotlib inline
plt.pyplot.style.use('ggplot')

### 2. Setting a seed for random number generators

In [116]:
# Setting the seed for random, so the experiments could be replicated
seed = 123
random.seed(seed)
np.random.seed(seed)

### 3. Defining time variables

In [117]:
# Defining a start time, to keep a check of time
start_time = time.time()

# Elapsed time can be calculated later as,
### elapsed_time = time.time() - start_time

### 4. Paramters for the network, such as constants and initial energy

In [129]:
# Defining the parameters

# Transmission Radius - 50m (meters)
R_o = 50

# Packet Lenth - 2000 bits
l = 2000

# Initial Energy for nodes - 0.5J (Joule)
E_o = 0.5


# Energy consumption on circuit - 50 nJ/bit
E_elec = 50/(10**9)

# Free-space channel parameter - 10 pJ/bit/m^2
E_fs = 10/(10**12)

# Multi-path channel parameter - 0.0013 pJ/bit/m^4
E_mp = 0.0013/(10**12)

# Distance Threshhold - (E_fs / E_mp) ^ 1/2 m
d_o = np.sqrt(E_fs/E_mp)

# Pheromone parameters
alpha = 1
beta = 3
p = 0.5 #Pheromone Evaporation Rate

# Energy consumed to receive a l-bit message
E_r_x = l*E_elec

# Round number
r = 0

# Upper limit for rounds, it could be set to way high, but for implementation purposes we take it to be 10
R = 10

# Desired percentage of cluster heads in a network, we can test it for different values, we take initially 0.25
per = 0.25

# Number of clusters, which we use in KMeans Clustering
num_clusters = 5

### 5. Defining the nodes, cluster head lists and visted lists

In [130]:
# Creating number of nodes by random
# Between 50 and 200
N = random.randint(50,201)
N

129

In [131]:
# Choosing locations for N nodes in an area of 100x100 m^2 randomly
# We will choose random numbers for each node's (x,y) co-ordinates between 0 to 100
# Creating empty list for the co-ordinates of each node
loc = []

# We declare co-ordinates for each node and also initial energy of 0.5 J
# IMPORTANT NOTE: The module, np.random will fail here, as the ACO pants performs a 'bool' operation internally
### Details at: https://stackoverflow.com/questions/10062954/valueerror-the-truth-value-of-an-array-with-more-than-one-element-is-ambiguous
for i in range(N):
    x = random.uniform(0,100)
    y = random.uniform(0,100)
    loc.append((x,y))

In [132]:
# Creating an energy array for all the nodes
E_current = np.empty(N)
E_current.fill(E_o)
print(E_current)

[ 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5]


In [133]:
# Cluster head history for num_clusters for (1/per) rounds
### e.g. in this case, for 5 clusters for 4 rounds
cluster_head = np.empty((num_clusters,int(1/per)))

# We initialize with -1 and update the cluster head list
cluster_head.fill(-1)
print(cluster_head)

[[-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]]


### 6. Functions to be used based on the theory

In [134]:
# Euclidean Distance Function
def dist(a,b):
    '''
    Returns the euclidean distance between two nodes
    
    Input::
    Args:
    a:: np.ndarray, containing node location and energy
    b:: np.ndarray, containing node location and energy
    
    
    Returns:
    d:: float, euclidean distance between two nodes
    
    '''
    d = math.sqrt(pow((a[1]-b[1]),2)+pow((a[0]-b[0]),2))
    
    return d
    
    
# Heuristic Desirability, N_i_j
def hue_des(d):
    '''
    Returns the heuristic desirability
    
    Input::
    Args:
    d:: float, eucledian distance between two nodes
    
    Returns::
    (d**(-1)):: float, the heuristic desirability
    '''
    return pow(d,-1)

# Energy Spent in transmission of l-bit message over distance d
def E_t_x(d):
    '''
    Returns the energy spent to transmit a l-bit message over distance d
    
    Input::
    Args:
    d:: float, euclidean distance between two nodes
    
    Returns::
    etx:: float, energy transmitted based on the distance    
    '''
    if (d<d_o):
        etx = l*E_elec + l*E_fs*pow(d,2)
    else:
        etx = l*E_elec + l*E_mp*pow(d,4)
    return etx

def predefined_threshold(a,c):
    '''
    Returns the predefined threshold based on the whether the node has been selected as clusted head in last 1/p rounds
    
    Input::
    Args:
    a:: int, the index of the node for which the threshold distance has to be calculated
    c:: int, cluster number
    
    Returns::
    thres:: float, threshold value for the node    
    '''
    # If the node has not been selected as a cluster head for last (1/per) rounds
    if (a not in cluster_head[c]):
        thres = (per/(1-(per*(r % (1/per)))))*(E_current[a]/E_o)
    else:
        thres = 0
    return thres
    


### 7. Implementing the ACO
#### Although we defined, alpha and beta for this algorithm, this implementation is using those values as default
#### To change these values refer to the link provided in the second reference at top

In [135]:
# Here we use KMeans clustering to classify the nodes into different clusters
kmeans = KMeans(n_clusters = num_clusters, random_state =0).fit_predict(loc)
print(kmeans)

[2 2 2 4 1 1 0 0 3 3 4 3 0 3 4 1 1 0 2 2 0 1 1 3 1 4 1 3 4 0 4 0 3 2 0 4 0
 0 2 2 3 0 1 4 2 3 4 1 0 1 1 0 1 0 2 3 4 2 0 3 2 1 3 4 4 3 1 2 0 0 4 4 3 3
 2 2 2 2 4 4 2 2 3 2 0 4 2 1 0 3 4 3 1 3 4 2 3 3 2 4 2 1 0 4 3 3 3 1 2 4 4
 3 0 0 4 4 3 3 2 1 4 0 4 3 0 3 1 0 2]


In [136]:
# Number of nodes in different clusters
co = collections.Counter(kmeans)
print(co)

Counter({3: 29, 2: 27, 4: 27, 0: 25, 1: 21})


In [137]:
# Getting indices of nodes for different clusters
li = []

for i in range(num_clusters):
    li.append(np.where(kmeans==i)[0])
print(li)

[array([  6,   7,  12,  17,  20,  29,  31,  34,  36,  37,  41,  48,  51,
        53,  58,  68,  69,  84,  88, 102, 112, 113, 121, 124, 127]), array([  4,   5,  15,  16,  21,  22,  24,  26,  42,  47,  49,  50,  52,
        61,  66,  87,  92, 101, 107, 119, 126]), array([  0,   1,   2,  18,  19,  33,  38,  39,  44,  54,  57,  60,  67,
        74,  75,  76,  77,  80,  81,  83,  86,  95,  98, 100, 108, 118, 128]), array([  8,   9,  11,  13,  23,  27,  32,  40,  45,  55,  59,  62,  65,
        72,  73,  82,  89,  91,  93,  96,  97, 104, 105, 106, 111, 116,
       117, 123, 125]), array([  3,  10,  14,  25,  28,  30,  35,  43,  46,  56,  63,  64,  70,
        71,  78,  79,  85,  90,  94,  99, 103, 109, 110, 114, 115, 120, 122])]


In [138]:
# The list for cluster heads to be optimized in each round
curr_cluster_heads = np.zeros([num_clusters,2])
print(curr_cluster_heads)

# Temporary cluster head index for each cluster, to be used as a dummy max till cluster head is finalized
curr_max = np.empty(num_clusters)
curr_max.fill(-1)
print(curr_max)

[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]
[-1. -1. -1. -1. -1.]


In [139]:
# Running the ACO for R rounds
for i in range(1,(R+1)):
    print("Round number"+str(i))
    # Random Number generation by each node
    for j in range(N):
        # Random number for each node to decide for cluster head
        check = random.uniform(0,1)
        
        # Comparing it to the threshold
        # kmeans[j] is the value of cluster the jth node belongs to
        if((check<predefined_threshold(j,kmeans[j]))):
            if(curr_max[kmeans[j]]==(-1)):
                curr_max[kmeans[j]]=j
            elif(E_current[j]>E_current[int(curr_max[kmeans[j]])]):
                curr_max[kmeans[j]]=j
           
           
    for j in range(num_clusters):
        curr_cluster_heads[j]=[loc[int(curr_max[j])][0] ,loc[int(curr_max[j])][1]]
        cluster_head[j][int((i-1)%int(1/per))]= curr_max[j]
            
        
                
                
        
                
    # NOTE: Each round, the nodes are varied for the ACO, as the cluster heads changes        
    world = pants.World(curr_cluster_heads.tolist(), dist) 
    
    # Implementing the ACO
    solver = pants.Solver()
    solution = solver.solve(world)
    
    # Distance travelled
    print('Distance for Round: '+str(i)+' : '+ str(solution.distance))
    
    # Nodes visited in order
    print(solution.tour) 
    
    # Edges taken in order
    print(solution.path)
    
    # Time Elapsed
    print('Time taken: '+str((time.time()-start_time)))
           
    # Energy Updates
    for j in range(N):
        # If the current node is the cluster head, it will loose energy for receiving messages from all other
        # That is number of nodes in that cluster-1
        if(j==int(curr_max[kmeans[j]])):
            E_current[j] = E_current[j]-(E_r_x*(co[kmeans[j]]-1))
    
        # Otherwise the energy is lost to transmit the 'l-bit' message to the cluster head
        else:
            E_current[j] = E_current[j]-(E_t_x(dist(loc[j],curr_cluster_heads[kmeans[j]])))
        
    
    # Energy left for nodes
    print('Current Energy\n'+str(E_current))r=r+1

Round number1
Distance for Round: 1 : 269.84351028732925
[[30.605622013958854, 7.8662011695320615], [2.0666747680248343, 81.88759803433672], [77.32137114957902, 85.91475266349227], [43.4721582057631, 50.63849671730294], [60.89665607968844, 23.70855872383234]]
[<pants.world.Edge object at 0x7f5b27841ef0>, <pants.world.Edge object at 0x7f5b27841080>, <pants.world.Edge object at 0x7f5b27841160>, <pants.world.Edge object at 0x7f5b27841fd0>, <pants.world.Edge object at 0x7f5b27841dd8>]
Time taken: 59.92111420631409
Current Energy
[ 0.4974      0.49989902  0.49989231  0.4974      0.49989016  0.498
  0.499893    0.4998951   0.4972      0.49989997  0.4998712   0.49989597
  0.49988348  0.49989597  0.4998672   0.4998993   0.49989085  0.4998781
  0.4998837   0.49989458  0.4976      0.49988678  0.49986489  0.49989284
  0.49989725  0.49989573  0.49989619  0.4998998   0.49989817  0.49989471
  0.49989668  0.49987244  0.49988712  0.49988714  0.49987008  0.49986578
  0.49989391  0.49989954  0.49989528 

In [148]:
# Testing The Values after the iterations
print('Current cluster heads:\n'+str(curr_cluster_heads))
print('Current candidates for clusters:\n'+str(curr_max))
print('Clusters Heads for last (1/per) rounds\n'+str(cluster_head))
print('Current Energy Levels:\n'+str(E_current))



Current cluster heads:
[[ 12.97511991  69.82979177]
 [ 62.81996817  10.20885166]
 [ 79.42010502  55.71100783]
 [ 69.91355043  80.21704245]
 [  8.83979644  26.1823022 ]]
Current candidates for clusters:
[ 29.  42.  54.  72.  99.]
Clusters Heads for last (1/per) rounds
[[  58.   29.   84.   48.]
 [ 119.   42.   87.   47.]
 [  67.   54.   57.   19.]
 [  55.   72.   73.   97.]
 [  30.   99.   63.   43.]]
Current Energy Levels:
[ 0.49642885  0.4964495   0.4989016   0.49643207  0.49892764  0.49707786
  0.4989553   0.49896876  0.49629228  0.49899216  0.49879881  0.49894194
  0.49888798  0.49894504  0.49878548  0.4970887   0.49892873  0.49883818
  0.49893496  0.49645107  0.49667521  0.49885929  0.49872238  0.49893083
  0.49708448  0.4964594   0.49894521  0.49629195  0.49645724  0.49663806
  0.49642608  0.49880045  0.49885372  0.49892592  0.49877679  0.49876599
  0.49889321  0.49668346  0.49892081  0.49864569  0.49896988  0.49883185
  0.49708045  0.49636768  0.49881012  0.49896059  0.49883545  